In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import scipy.sparse as sps
import scipy.sparse.linalg
from scipy.special.orthogonal import p_roots
from pyamg.classical import ruge_stuben_solver

In [ ]:
from topo import SQuad
from poly import lagrange_list
from basis import LagrangeBasisQuad
from mesh import Mesh2D
from assemble import simple_assembly, simple_build_rhs

## Mesh nodes

In [ ]:
order   = 2
L       = 1.0
n_elems = 16

x_max = L
y_max = L
x_vals = np.linspace(0, x_max, n_elems+1)
y_vals = np.linspace(0, y_max, n_elems+1)

vertices = np.zeros(((n_elems+1)**2, 2), dtype=np.double)
elem_to_vertex = np.zeros((n_elems**2, 4), dtype=np.int)

for i in range(n_elems):
    for j in range(n_elems):
        elem = i*n_elems+j
        elem_to_vertex[elem,0] = i*(n_elems+1)+j
        elem_to_vertex[elem,1] = i*(n_elems+1)+j+1
        elem_to_vertex[elem,2] = (i+1)*(n_elems+1)+j+1
        elem_to_vertex[elem,3] = (i+1)*(n_elems+1)+j

boundary_vertices = []
for i in range(n_elems+1):
    for j in range(n_elems+1):
        v = i*(n_elems+1)+j
        vertices[v,0] = x_vals[j]
        vertices[v,1] = y_vals[i]
        if (i==0) or (j==0) or\
           (i==n_elems) or (j==n_elems):
            boundary_vertices.append(v)


## Assembly

In [ ]:
topo  = SQuad()
basis = LagrangeBasisQuad(SQuad, order)
mesh  = Mesh2D(topo, basis)
mesh.build_mesh(vertices, elem_to_vertex, boundary_vertices)

nodes = vertices[elem_to_vertex]
jacb  = topo.calc_jacb(nodes)
jacb_det = topo.calc_jacb_det(jacb)
jacb_inv = topo.calc_jacb_inv(jacb)
jacb = jacb[0]
jacb_det = jacb_det[0]
jacb_inv = jacb_inv[0]

In [ ]:
cub_points, cub_weights = topo.get_quadrature(order+1)
Kloc = np.zeros((basis.n_dofs, basis.n_dofs),
                dtype=np.double)
cub_vals = basis.eval_ref(np.eye(basis.n_dofs),
                          cub_points, d=1)

for i in range(basis.n_dofs):
    for j in range(basis.n_dofs):
        d1 = jacb_inv.T.dot(cub_vals[i])
        d2 = jacb_inv.T.dot(cub_vals[j])
        p = d1*d2
        Kloc[i,j] = np.sum(p, axis=0).dot(cub_weights)
Kloc = jacb_det*Kloc

K = simple_assembly(mesh, Kloc)

def f(X):
    shape = X.shape[:-1]
    X = X.reshape((-1,2))
    x = X[:,0]
    y = X[:,1]
    return (x*(x-x_max)*y*(y-y_max)).reshape(shape)

def f2(X):
    shape = X.shape[:-1]
    X = X.reshape((-1,2))
    x = X[:,0]
    y = X[:,1]
    return (2*y*(y-y_max)+x*(x-x_max)*2).reshape(shape)

rhs = simple_build_rhs(topo, basis, mesh, f2)

In [ ]:
plt.spy(K)
(K-K.T).nnz

In [ ]:
ml = ruge_stuben_solver(K)
residuals = []
sol = ml.solve(rhs, tol=1e-12, residuals=residuals, maxiter=5000)
len(residuals), residuals[-1]

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

X = vertices[:,0].reshape((n_elems+1, n_elems+1))
Y = vertices[:,1].reshape((n_elems+1, n_elems+1))
Z1 = sol[mesh.vertex_to_dof].ravel()
Z2 = f(mesh.vertices).ravel()
ax.plot_wireframe(X, Y, Z1.reshape((n_elems+1, n_elems+1)))
ax.plot_wireframe(X, Y, Z2.reshape((n_elems+1, n_elems+1)), 
                  color='g')
np.max(np.abs(Z1-Z2))